# Lab: Even More Linear Regression
## CMSE 381 - Fall 2022
## Lecture 7, Sep 16, 2022

In the last few lectures, we have focused on linear regression, that is, fitting models of the form 
$$
Y =  \beta_0 +  \beta_1 X_1 +  \beta_2 X_2 + \cdots +  \beta_pX_p + \varepsilon
$$
In this lab, we will continue to use two different tools for linear regression. 
- [Scikit learn](https://scikit-learn.org/stable/index.html) is arguably the most used tool for machine learning in python 
- [Statsmodels](https://www.statsmodels.org) provides many of the statisitcial tests we've been learning in class

This lab will cover two ideas: 
- Categorical variables and how to represent them as dummy variables. 
- How to build interaction terms and pass them into your favorite model.

In [11]:
# As always, we start with our favorite standard imports. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as smf

# Playing with multi-level variables 

## The wrong way

Ok, we're going to do this incorrectly to start. You pull in the `Auto` data set. You were so proud of yourself for remembering to fix the problems with the `horsepower` column that you conveniently forgot that the column with information about country of origin (`origin`) has a bunch of integers in it, representing:
- 1: `American`
- 2: `European`
- 3: `Japanese`.

In [12]:
Auto_df = pd.read_csv('Auto.csv')
Auto_df = Auto_df.replace('?', np.nan)
Auto_df = Auto_df.dropna()
Auto_df.horsepower = Auto_df.horsepower.astype('int')


Auto_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')

You then go on your merry way building the model 
$$
\texttt{mpg} = \beta_0 + \beta_1 \cdot \texttt{origin}. 
$$

In [13]:
from sklearn.linear_model import LinearRegression

X = Auto_df.origin.values
X = X.reshape(-1, 1)
y = Auto_df.mpg.values

regr = LinearRegression()

regr.fit(X,y)

print('beta_1 = ', regr.coef_[0])
print('beta_0 = ', regr.intercept_)

beta_1 =  5.476547480191448
beta_0 =  14.81197361541246


&#9989; **<font color=red>Q:</font>** What does your model predict for each of the three types of cars? 

In [14]:
beta_0 = 14.81197361541246
beta_1 = 5.476547480191448

# Origin values
origins = {
    "American": 1,
    "European": 2,
    "Japanese": 3
}

# Predict MPG for each origin
for origin, value in origins.items():
    mpg = beta_0 + beta_1 * value
    print(f"{origin}: Predicted MPG = {mpg:.2f}")

American: Predicted MPG = 20.29
European: Predicted MPG = 25.77
Japanese: Predicted MPG = 31.24


&#9989; **<font color=red>Q:</font>** Is it possible for your model to predict that both American and Japanese cars have `mpg` below European cars? 

**No, it's not possible with this model—mpg increases as origin increases, so Japanese cars always have the highest predicted mpg, and American the lowest.**

## The right way

Ok, so you figure out your problem and decide to load in your data and fix the `origin` column to have names as entries.

In [15]:
convertOrigin= {1: 'American', 2:'European', 3:'Japanese'}

# This command swaps out each number n for convertOrigin[n], making it one of
# the three strings instead of an integer now.
Auto_df.origin = Auto_df.origin.apply(lambda n: convertOrigin[n])
Auto_df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,American,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,American,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,American,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,American,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,American,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86,2790,15.6,82,American,ford mustang gl
393,44.0,4,97.0,52,2130,24.6,82,European,vw pickup
394,32.0,4,135.0,84,2295,11.6,82,American,dodge rampage
395,28.0,4,120.0,79,2625,18.6,82,American,ford ranger


Below is a quick code that automatically generates our dummy variables. Yay for not having to code that mess ourselves!

In [16]:
origin_dummies_df = pd.get_dummies(Auto_df.origin, prefix='origin')
origin_dummies_df

,origin_American,origin_European,origin_Japanese
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
392,True,False,False
393,False,True,False
394,True,False,False
395,True,False,False


&#9989; **<font color=red>Q:</font>** What is the interpretation of each column in the `origin_dummies_df` data frame?

Each column in origin_dummies_df represents a binary dummy variable for a car origin. 

For example:
- origin_American is 1 if the car is American, else 0
- origin_European is 1 if European, else 0
- origin_Japanese is 1 if Japanese, else 0

Each row will have a 1 in exactly one of the three columns.

I pass these new dummy variables into my `scikit-learn` linear regression model and get the following coefficients

In [17]:
X = origin_dummies_df.values
y = Auto_df.mpg

regr = LinearRegression()

regr.fit(X,y)

print('Coefs = ', regr.coef_)
print('Intercept = ', regr.intercept_)

Coefs =  [-5.9955451   1.57392668  4.42161842]
Intercept =  26.0290144918727


&#9989; **<font color=red>Q:</font>** Now what does your model predict for each of the three types of cars? 

In [18]:
intercept = 26.0290144918727
coefs = {
    'American': -5.9955451,
    'European': 1.57392668,
    'Japanese': 4.42161842
}

# Predict mpg for each car origin
for origin in ['American', 'European', 'Japanese']:
    mpg = intercept + coefs[origin]
    print(f"{origin}: Predicted MPG = {mpg:.2f}")

American: Predicted MPG = 20.03
European: Predicted MPG = 27.60
Japanese: Predicted MPG = 30.45


### Ooops

&#9989; **<font color=red>Q:</font>** Aw man, I didn't quite do what we said for the dummy variables in class. We talked about having only two dummy variables for a three level variable. Copy my code below here and fix it to have two variables instead of three. 
- Are your coefficients different now?
- Are your predictions for each of the three origins different now? 
- Does it matter which two levels you used for your dummy variables? 

In [19]:
from sklearn.linear_model import LinearRegression

convertOrigin = {1: 'American', 2: 'European', 3: 'Japanese'}
Auto_df['origin'] = Auto_df['origin'].replace(convertOrigin)

Auto_df = pd.get_dummies(Auto_df, columns=['origin'], drop_first=True)

X = Auto_df[['origin_European', 'origin_Japanese']]  # American is baseline
y = Auto_df['mpg']

regr = LinearRegression()
regr.fit(X, y)

print("Coefs =", regr.coef_)
print("Intercept =", regr.intercept_)

Coefs = [ 7.56947179 10.41716352]
Intercept = 20.0334693877551


## Another right way

Ok, fine, I'll cave, I made you do it the hard way but you got to see how the innards worked, so maybe it's not all bad ;) 

The following code does the same thing as above, but because `statsmodels` has built in tools to handle categorical variables in a data frame, it does the hard work for you. 

In [22]:
est = smf.ols('mpg ~ origin', Auto_df).fit()
est.summary().tables[1]

PatsyError: Number of rows mismatch between data argument and origin (392 versus 1)
    mpg ~ origin
          ^^^^^^

&#9989; **<font color=red>Q:</font>** What is the model learned from the above printout? Be specific in terms of your dummy variables. 

Your answer here

![Stop Icon](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Vienna_Convention_road_sign_B2a.svg/180px-Vienna_Convention_road_sign_B2a.svg.png)

Great, you got to here! Hang out for a bit, there's more lecture before we go on to the next portion. 

# Interaction Terms 

Below is the code I have for generating the tables shown on the slides. 

In [23]:
Advertising_df = pd.read_csv('Advertising.csv', index_col = 0)
Advertising_df.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [24]:
Advertising_df['TVxRadio'] = Advertising_df.TV * Advertising_df.Radio
Advertising_df.head()

,TV,Radio,Newspaper,Sales,TVxRadio
1,230.1,37.8,69.2,22.1,8697.78
2,44.5,39.3,45.1,10.4,1748.85
3,17.2,45.9,69.3,9.3,789.48
4,151.5,41.3,58.5,18.5,6256.95
5,180.8,10.8,58.4,12.9,1952.64


In [25]:
est = smf.ols('Sales ~ TV + Radio + TVxRadio', Advertising_df).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,6.7502,0.248,27.233,0.000,6.261,7.239
TV,0.0191,0.002,12.699,0.000,0.016,0.022
Radio,0.0289,0.009,3.241,0.001,0.011,0.046
TVxRadio,0.0011,5.24e-05,20.727,0.000,0.001,0.001


&#9989; **<font color=red>Do this:</font>** Using the Auto data set, train the model 
$$
\texttt{mpg} = \beta_0 + \beta_1\cdot \texttt{weight} + \beta_2\cdot \texttt{horsepower} + \beta_3\cdot \texttt{weight x horsepower}.
$$
Is the interaction term adding value to the model? 

In [27]:
# Create interaction term
Auto_df['weight_x_horsepower'] = Auto_df['weight'] * Auto_df['horsepower']

# Train the model with interaction term
model = smf.ols('mpg ~ weight + horsepower + weight_x_horsepower', data=Auto_df).fit()

# Show coefficient summary
print(model.summary().tables[1])

                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              63.5579      2.343     27.127      0.000      58.951      68.164
weight                 -0.0108      0.001    -13.921      0.000      -0.012      -0.009
horsepower             -0.2508      0.027     -9.195      0.000      -0.304      -0.197
weight_x_horsepower  5.355e-05   6.65e-06      8.054      0.000    4.05e-05    6.66e-05


 Yes, the interaction term is adding value — its p-value is 0.000, which is highly significant.



-----
### Congratulations, we're done!

Written by Dr. Liz Munch, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.